In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
from tools.extensions import AnnotationTask

/data/anaconda/envs/py37torch/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [19]:
df = pd.read_csv('output_data/MPD_export_1846_1920.csv', index_col=0)


In [27]:
task.df[4000:4005]

,index,id,S-TITLE,S-POL,CATEGORY,DISTRICT,DISTRICT_PUB,COUNTY,S-PRICE,D-EST,D-PUB,E-LOC,E-ORG,E-PER,S-TITLE-ALT,TEXT,DISTRICT_DESCRIPTION,year,chain_id
4000,707,MPD_1860_707,WESTERN FLYING POST,conservative,provincial,yeovil . — ( somersetshire . ),yeovil,somersetshire,4d<SEP>5d,1736,Tuesday,Somerset<SEP>Dorset<SEP>Wilts<SEP>Devon<SEP>Co...,NaN,James Jarvis,NaN,,NaN,1860,CID_000198
4001,708,MPD_1860_708,YORK HERALD .,liberal,provincial,york . — ( yorkshire . ),york,yorkshire,3½d<SEP>4½d,1790,Saturday,York<SEP>Durham<SEP>Cumberland<SEP>Northumberl...,NaN,Alfred Ely Hargrove<SEP>William Wallace Hargrove,NaN,,NaN,1860,CID_000613
4002,709,MPD_1860_709,YORKSHIRE GAZETTE .,conservative,provincial,york . — ( yorkshire . ),york,yorkshire,3½d<SEP>4½d,1819,Saturday,York<SEP>North and East Ridings,NaN,J . Lancelot Foster,NaN,"VYOWRON . Saturday , Price 2 ½ d . and 3 ½ d ....",This is a large and populous parish in the cou...,1860,CID_000316
4003,710,MPD_1860_710,RACING CALENDER ( JOHNSON'S ) .,neutral,provincial,york . — ( yorkshire . ),york,yorkshire,one guinea,1821,Weekly,NaN,NaN,R . Johnson,NaN,,NaN,1860,CID_002275
4004,711,MPD_1860_711,VYOWRON .,liberal,welsh,aberdare . — ( glamorganshire . ),aberdare,glamorganshire,2 ½ d<SEP>3 ½ d<SEP>1d,1851,Saturday,Glamorganshire,NaN,Josiah Thomas Jones,NaN,NORTH WALES CHRONICLE AND ADVER - — TISER FOR ...,; This is a delightfully situate and improving...,1860,NaN


# Create data for annotation in INCEpTION

In [26]:
task_name = 'description'
task = AnnotationTask(df,task_name)
task.sample(n=100)
task.export_sample()


/data/anaconda/envs/py37torch/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Load annotations and train a model

In [21]:
task.create_training_data(clip_bioes=True)



51 62 74


In [23]:
task.train_model()

## Apply model and export

In [28]:
task.apply_model(Path('annotations/description/model/tagger/best-model.pt'))

2021-04-28 13:29:13,127 loading file annotations/description/model/tagger/best-model.pt
  9%|▉         | 4001/43361 [06:14<47:05, 13.93it/s]2021-04-28 13:35:28,770 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-04-28 13:35:28,771 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-04-28 13:35:28,830 Warning: An empty Sentence was created! Are there empty strings in your dataset?
 12%|█▏        | 5148/43361 [07:44<44:28, 14.32it/s]2021-04-28 13:36:58,760 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-04-28 13:36:58,761 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-04-28 13:36:58,762 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-04-28 13:36:58,763 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-04-28 13:36:58,764 Warning: An empty Sentence was created! Are there

In [29]:
task.extract_tag('Description')

In [30]:
task.save()

# Appendix
## Output Descriptions for STM

In [5]:
df_task = pd.read_csv('output_data/MPD_export_1846_1920_description.csv', index_col=0)

df_jisc = pd.read_csv('output_data/MPD_export_1846_1920_JISC.csv', index_col=0)
df_merged = df_task.merge(df_jisc['JISC'],left_index=True,right_index=True)
df_merged.shape

(43361, 21)

In [6]:
df_merged.loc[~df_merged.JISC.isnull(),'JISC'] = 1
df_merged['JISC'].fillna(0,inplace=True)


In [7]:
out_stm = Path('output_data/stm')
out_stm.mkdir(exist_ok=True)

In [9]:
df_merged[['year','COUNTY','JISC','S-POL','extract_description']].to_csv(out_stm / 'MPD_export_1846_1920_stm_jisc_export.csv' )

In [42]:

df_merged_sub = df_merged[df_merged['S-POL'].isin(['liberal', 'conservative','independent','neutral'])]
df_merged_sub[['year','COUNTY','JISC','S-POL','extract_description']].to_csv(out_stm / 'MPD_export_1846_1920_stm_pol_export.csv' )